In [36]:
import ee
import geemap
import os
import shapely
import json
import mgrs
import pandas as pd

In [79]:
Map = geemap.Map()

In [80]:
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [57]:
def import_aois(csv_loc):    

        df_labels = pd.read_csv(csv_loc)
        df_labels = df_labels[["center-lat","center-long","polygon","Labels combined"]]

        polygons = []
        for polygon in df_labels["polygon"]:
            polygons.append(json.loads(polygon)["coordinates"])


        tiles = []
        tiles_dic = {}
        polygon_id = 0 
        coordinates = []
        m = mgrs.MGRS()
        for items in polygons:
            polygon_id += 1 
            for item in items:
                for lon_lat in item:
                    coordinates.append(lon_lat)
                    c = m.toMGRS(lon_lat[1], lon_lat[0])
                    tile = c[0:5]
                    

                    if polygon_id in tiles_dic:

                        tiles_dic[polygon_id].append(tile)

                    else:

                        tiles_dic[polygon_id] = [tile]

                    tiles.append(tile)

                tiles_dic[polygon_id] = list(set(tiles_dic[polygon_id]))

        tiles = list(set(tiles))

        df_labels["tiles"] = tiles_dic.values()

        #bounding box

        min_lon = min([i[0] for i in coordinates])
        min_lat = min([i[1] for i in coordinates])
        max_lon = max([i[0] for i in coordinates])
        max_lat = max([i[1] for i in coordinates])

        bounding_box = min_lon,min_lat,max_lon,max_lat
        
        return polygons

In [58]:
polygons = import_aois("/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v3/Polygon_List/polygons_101320.csv")

In [60]:
polygons[0]

[[[9.088783, 5.753152],
  [9.179249, 5.703106],
  [9.248257, 5.792946],
  [9.236069, 5.845887],
  [9.090672, 5.844009],
  [9.088783, 5.753152]]]

In [6]:
ee.Initialize()

In [62]:
# region of interest
geom = ee.Geometry.Polygon(polygons[0])



In [78]:
# help(ee.ImageCollection)

In [65]:
# image collection
ic = ee.ImageCollection('COPERNICUS/S2')\
    .filterBounds(geom)\
    .filterDate(startDate,stopDate)
    
# # single image
# img = ee.Image(ic.first())

In [74]:
# ee.Image(ic.get(0)).getInfo()

In [94]:
# start and end of time series
startDate = ee.Date('2019-01-01')
stopDate  = ee.Date('2020-12-31')

# Add Earth Engine dataset
# Sentinel 2 ImageCollection
# filter by bounds and data
collection = ee.ImageCollection('COPERNICUS/S2')\
    .filterBounds(geom)\
    .filterDate(startDate,stopDate)\
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",5))

# # create an roi. first item in Misha's label list
polys = ee.Geometry.Polygon(polygons[0])

# # create a FeatureCollection based on the roi and center the map
centroid = polys.centroid()
lng, lat = centroid.getInfo()['coordinates']
print("lng = {}, lat = {}".format(lng, lat))
Map.setCenter(lng, lat, 12)
fc = ee.FeatureCollection(polys)

# # # filter the ImageCollection using the roi
mosaic = collection.mosaic()

# # # print out the number of images in the ImageCollection
count = collection.size().getInfo()
print("Count: ", count)

# add the ImageCollection and the roi to the map
vis = {'bands': ["B4", "B3", "B2"]}
Map.addLayer(mosaic,vis)
Map.addLayer(fc)

# # export the ImageCollection to Google Drive
# # downConfig = {'scale': 30, "maxPixels": 1.0E13, 'driveFolder': 'image'}  # scale means resolution.
img_lst = collection.toList(100)

# python.dataScience.textOutputLimit == "0"

for i in range(0, count):
    image = ee.Image(img_lst.get(i))
    print(image.bandNames())
    # name = image.get('system:time_start').getInfo()
#     name = image.get('CLOUDY_PIXEL_PERCENTAGE').getInfo()
    # print()
#     # task = ee.batch.Export.image(image, name, downConfig)
#     # task.start()

01"
                                                }
                                              }
                                            }
                                          }
                                        }
                                      }
                                    },
                                    "rightField": {
                                      "constantValue": "system:time_start"
                                    }
                                  }
                                }
                              }
                            }
                          }
                        },
                        "filter": {
                          "functionInvocationValue": {
                            "functionName": "Filter.lessThan",
                            "arguments": {
                              "leftField": {
                                "constantValue": "CLOUDY_PIXEL_PERCENTAGE"
                  

In [89]:
help(image.get)

Help on method Element.get in Element:

Element.get(*args, **kwargs) method of ee.image.Image instance
    Extract a property from a feature.
    
    Args:
      object: The feature to extract the property from.
      property: The property to extract.



In [27]:
# Add Earth Engine dataset
# USDA NAIP ImageCollection
collection = ee.ImageCollection('COPERNICUS/S2')

# create an roi
polys = ee.Geometry.Polygon(
        [[[-99.29615020751953, 46.725459351792374],
          [-99.2116928100586, 46.72404725733022],
          [-99.21443939208984, 46.772037733479884],
          [-99.30267333984375, 46.77321343419932]]])

# create a FeatureCollection based on the roi and center the map
centroid = polys.centroid()
lng, lat = centroid.getInfo()['coordinates']
print("lng = {}, lat = {}".format(lng, lat))
Map.setCenter(lng, lat, 12)
fc = ee.FeatureCollection(polys)

# filter the ImageCollection using the roi
naip = collection.filterBounds(polys)
naip_2015 = naip.filterDate('2020-01-01', '2020-12-31')
mosaic = naip_2015.mosaic()

# print out the number of images in the ImageCollection
count = naip_2015.size().getInfo()
print("Count: ", count)

# add the ImageCollection and the roi to the map
vis = {'bands': ["B4", "B3", "B2"]}
Map.addLayer(mosaic,vis)
Map.addLayer(fc)

# export the ImageCollection to Google Drive
# downConfig = {'scale': 30, "maxPixels": 1.0E13, 'driveFolder': 'image'}  # scale means resolution.
img_lst = naip_2015.toList(100)

for i in range(0, count):
    image = ee.Image(img_lst.get(i))
    # name = image.get('system:time_start').getInfo()
    name = image.get('CLOUDY_PIXEL_PERCENTAGE').getInfo()
    print(name)
    # task = ee.batch.Export.image(image, name, downConfig)
    # task.start()

lng = -99.25621066337796, lat = 46.748869171015414
Count:  121
64.6702
21.719
0.0727
99.996
98.4652
1.1837
94.7315
0.3224
99.8241
97.2318
94.0236
97.7256
100
99.7998
0.0447
13.0957
7.6933
99.989
37.6963
84.8453
4.7692
0.3775
78.8385
88.8554
70.4847
99.8974
1.7943
24.5385
98.2717
74.3904
95.9124
99.5793
74.7282
95.8228
37.2046
0
92.9287
97.9489
96.8831
13.492
29.4348
92.6898
29.1265
63.1012
8.6172
84.2204
90.4664
98.616
6.9861
0.0145
99.5073
33.5855
79.6652
100
71.5375
93.1644
87.0731
29.6351
100
2.2313
2.4095
20.6282
13.5606
4.7546
99.7352
0.9146
65.2546
55.2592
21.5043
42.3724
0.0316
9.3962
61.5479
0.1854
66.8579
3.6689
39.8041
8.3088
0.4223
6.2606
0
0.2153
0.0718
28.1331
0.0412
42.4373
9.1495
0.8248
1.9919
86.5073
28.9231
20.4001
0
0.01
99.5643
23.5704
19.5614
3.0203
92.1422
11.858


EEException: List.get: List index must be between 0 and 99, or -100 and -1.  Found 100.

In [19]:
help(image.getInfo()))

TypeError: getInfo() takes 1 positional argument but 2 were given